# String Transformations

In [ ]:
!pip install more-polars

In [ ]:
!pip install more-polars --upgrade

## Outline

1. Basic string operations<br>
    a. Managing case and whitespace<br>
    b. Replacing substrings<br>
2. Regular expressions in Python
3. Using functions and RegEx with `df.column.replace`

In [1]:
import polars as pl
pl.Config.with_columns_kwargs = True

## Data set

We will be using two of the data sets provided by the Museam of Modern Art (MoMA) in this lecture.  Make sure that you have downloaded each repository.  [Download Instructions](./get_MOMA_data.ipynb)

#### MoMA Artists

In [2]:
artists = pl.read_csv("./data/Artists.csv")
artists.head(2)

ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
i64,str,str,str,str,i64,i64,str,i64
1,"""Robert Arneson...","""American, 1930...","""American""","""Male""",1930,1992,null,null
2,"""Doroteo Arnaiz...","""Spanish, born ...","""Spanish""","""Male""",1936,0,null,null


#### MoMA Artwork

In [3]:
artwork = pl.read_csv("./data/Artworks.csv")
          
artwork.head(2)

Title,Artist,ConstituentID,ArtistBio,Nationality,BeginDate,EndDate,Gender,Date,Medium,Dimensions,CreditLine,AccessionNumber,Classification,Department,DateAcquired,Cataloged,ObjectID,URL,ThumbnailURL,Circumference (cm),Depth (cm),Diameter (cm),Height (cm),Length (cm),Weight (kg),Width (cm),Seat Height (cm),Duration (sec.)
str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,str,str,str,str,str,f64,str,str,f64,str,str
"""Ferdinandsbrüc...","""Otto Wagner""","""6210""","""(Austrian, 184...","""(Austrian)""","""(1841)""","""(1918)""","""(Male)""","""1896""","""Ink and cut-an...","""19 1/8 x 66 1/...","""Fractional and...","""885.1996""","""Architecture""","""Architecture &...","""1996-04-09""","""Y""",2,"""http://www.mom...","""http://www.mom...",null,null,null,48.6,null,null,168.9,null,null
"""City of Music,...","""Christian de P...","""7470""","""(French, born ...","""(French)""","""(1944)""","""(0)""","""(Male)""","""1987""","""Paint and colo...","""16 x 11 3/4"" (...","""Gift of the ar...","""1.1995""","""Architecture""","""Architecture &...","""1995-01-17""","""Y""",3,"""http://www.mom...","""http://www.mom...",null,null,null,40.6401,null,null,29.8451,null,null


# Working with Strings

In [4]:
artists.head(2)

ConstituentID,DisplayName,ArtistBio,Nationality,Gender,BeginDate,EndDate,Wiki QID,ULAN
i64,str,str,str,str,i64,i64,str,i64
1,"""Robert Arneson...","""American, 1930...","""American""","""Male""",1930,1992,null,null
2,"""Doroteo Arnaiz...","""Spanish, born ...","""Spanish""","""Male""",1936,0,null,null


## Working with string  `polars`

* Available under `pl.col('col_name').str`
* Match most python methods

## Managing case and whitespace

The following table compare the functions/methods for managing case

| `python` method | `polars` column expression  |
| --- | --- |
| `s.lower()`  | `pl.col('a').str.to_lowercase()`   |
| `s.upper()`  | `pl.col('a').str.to_uppercase()`   |
| `s.strip()`  | `pl.col('a').str.strip()`   |
| `s.lstrip()` | `pl.col('a').str.lstrip()` |
| `s.rstrip()` | `pl.col('a').str.rstrip()` |

In [5]:
[m for m in dir(pl.col('DisplayName').str) if not m.startswith('_')]

['concat',
 'contains',
 'count_match',
 'decode',
 'encode',
 'ends_with',
 'extract',
 'extract_all',
 'json_path_match',
 'lengths',
 'ljust',
 'lstrip',
 'n_chars',
 'replace',
 'replace_all',
 'rjust',
 'rstrip',
 'slice',
 'split',
 'split_exact',
 'splitn',
 'starts_with',
 'strip',
 'strptime',
 'to_lowercase',
 'to_uppercase',
 'zfill']

## The missing `polars` string documentation.

As of Fall 2022, the documentation for `polars` string methods is lacking, and the best place to look is in the source code.  You can find the code for the string name space on the [polars GitHub site](https://github.com/pola-rs/polars), the string namespace is defined in the file `py-polars/polars/internals/series/string.py`.  Once there, use `CMD/CTRL + F` and search for the method of interest.

## <font color="red"> Exercise 3.1.1 </font>

Find the doc string for `to_lowercase` method, paste the multi-line string below, and then comment on what you learn.

In [6]:
# Doc string here -- copy all lines between the """ (inclusive)

> <font color="orange"> Tell me what you learned </font>

## Example - Lower-case Artists

In [7]:
artwork['Artist'].str.to_lowercase()

shape: (138151,)
Series: 'Artist' [str]
[
	"otto wagner"
	"christian de p...
	"emil hoppe"
	"bernard tschum...
	"emil hoppe"
	"bernard tschum...
	"bernard tschum...
	"bernard tschum...
	"bernard tschum...
	"bernard tschum...
	"bernard tschum...
	"bernard tschum...
	...
	"unknown photog...
	"unknown photog...
	"unknown photog...
	"unknown photog...
	"unknown photog...
	"unknown photog...
	"unknown photog...
	"unknown photog...
	"chesnutt broth...
	"sophie taeuber...
	"sophie taeuber...
	"sophie taeuber...
	"sophie taeuber...
]

#### Applying `str.to_lowercase` in `polar`

In [8]:
(artwork
 .select(['Artist'])
 .with_columns(Artist_lower = pl.col('Artist').str.to_lowercase())
 .head(2)
)

Artist,Artist_lower
str,str
"""Otto Wagner""","""otto wagner"""
"""Christian de P...","""christian de p..."


## Replacing a substring

Another important string operation involves replacing one substring with another.  In this section, we will illustrate using a regular expression to accomplish this task.

### Replace methods in `polars`

- `str.replace` replaces the first case.
- `str.replace_all` replace all matches
- Both use RegEx by default.
- Turn off RegEx using `literal=True`

## Example - The BeginDate mess

In [9]:
artwork.select(pl.col('BeginDate')).head()

BeginDate
str
"""(1841)"""
"""(1944)"""
"""(1876)"""
"""(1944)"""
"""(1876)"""


## Using `replace` to remove the first parenthesis

In [10]:
(artwork.select('BeginDate')
 .with_columns(BeginDate = pl.col('BeginDate')
                             .str.replace(r'[()]', ''))
 .head()
)

BeginDate
str
"""1841)"""
"""1944)"""
"""1876)"""
"""1944)"""
"""1876)"""


## Using `replace_all` to remove all left parenthesis

In [11]:
(artwork.select('BeginDate')
 .with_columns(BeginDate = pl.col('BeginDate')
                             .str.replace_all(r'[()]', ''))
 .head()
)

BeginDate
str
"""1841"""
"""1944"""
"""1876"""
"""1944"""
"""1876"""


# Splitting strings

* `SPLIT` - important string verb
* Use `str.split` to split on all instances.
* Other versions
    - `split_exact` to split exactly `n` times.
    - `splitn` to split up to `n` times.
* DOES NOT accept regular expressions
    - Need to combine with `replace_all`

### Example 1 - Splitting the Artists Name

In [12]:
artwork['Artist'].str.split(' ').head(2)

shape: (2,)
Series: 'Artist' [list]
[
	["Otto", "Wagner"]
	["Christian", "de", "Portzamparc"]
]

## `dtype: list` - more than just a string

* Splitting a string results in a `pl.Series` of ``dtype=list`. 
* Methods for working with a list are in the `arr` namespace

In [13]:
[m for m in dir(pl.col('*').arr) if not m.startswith('_')]

['arg_max',
 'arg_min',
 'concat',
 'contains',
 'diff',
 'eval',
 'first',
 'get',
 'head',
 'join',
 'last',
 'lengths',
 'max',
 'mean',
 'min',
 'reverse',
 'shift',
 'slice',
 'sort',
 'sum',
 'tail',
 'to_struct',
 'unique']

## Using `get` to get one part

Since `split` returns a list, we will need to use `arr.get` to pull out an entry.

In [14]:
artwork['Artist'].str.split(' ').arr.get(0).head()

shape: (10,)
Series: 'Artist' [str]
[
	"Otto"
	"Christian"
	"Emil"
	"Bernard"
	"Emil"
	"Bernard"
	"Bernard"
	"Bernard"
	"Bernard"
	"Bernard"
]

## Remember negative indexing!

* Negative index == counting from the right

In [15]:
artwork['Artist'].str.split(' ').arr.get(-1).head()

shape: (10,)
Series: 'Artist' [str]
[
	"Wagner"
	"Portzamparc"
	"Hoppe"
	"Tschumi"
	"Hoppe"
	"Tschumi"
	"Tschumi"
	"Tschumi"
	"Tschumi"
	"Tschumi"
]

### Example 2 - Splitting the Artists Bio

In [16]:
pl.Config.set_ascii_tables()
pl.Config.set_tbl_width_chars(264)
pl.Config.set_fmt_str_lengths(128)

polars.cfg.Config

In [17]:
(artwork
 .select(pl.col('ArtistBio'))
 .with_columns(ArtistBio = pl.col('ArtistBio').str.replace_all('[()]', ''))
 .with_columns(ArtistiBioNew = pl.col('ArtistBio')
                                 .str.replace_all(r' ,|, born |-|\s', '|')
                                 .str.split('|')#.arr.get(-1)
              )
).head()

ArtistBio,ArtistiBioNew
str,list[str]
"""Austrian, 1841–1918""","[""Austrian,"", ""1841–1918""]"
"""French, born 1944""","[""French"", ""1944""]"
"""Austrian, 1876–1957""","[""Austrian,"", ""1876–1957""]"
"""French and Swiss, born Switzerland 1944""","[""French"", ""and"", ... ""1944""]"
"""Austrian, 1876–1957""","[""Austrian,"", ""1876–1957""]"


## <font color="red"> Exercise 3.1.2 </font>

1. Why didn't splitting on `-` work in the last example?
2. See if you can fix this issue

> *Your thoughts here*

In [18]:
# Your code here

## Extracting a substring of a fixed size/position

In `polars`, use the `str.slice(offset)` method
* `offset` is the starting index
* Use `length=n` to slice up to `n` characters


In [20]:
s = pl.Series("s", ["pear", None, "papaya", "dragonfruit"])
s.str.slice(-3)

shape: (4,)
Series: 's' [str]
[
	"ear"
	null
	"aya"
	"uit"
]

In [21]:
s.str.slice(4, length=3)

shape: (4,)
Series: 's' [str]
[
	""
	null
	"ya"
	"onf"
]

In [22]:
(artwork
 .select('BeginDate')
 .with_columns(BeginDate = pl.col('BeginDate').str.replace_all('[()]', ''))
 .with_columns(century = pl.col('BeginDate').str.slice(0, length = 2),
               year_in_century = pl.col('BeginDate').str.slice(-2))
 .head(5))

BeginDate,century,year_in_century
str,str,str
"""1841""","""18""","""41"""
"""1944""","""19""","""44"""
"""1876""","""18""","""76"""
"""1944""","""19""","""44"""
"""1876""","""18""","""76"""


## Extracting a substring with RegEx

To extract a column in `polars` with RegEx

* Use `str.extract(pat)` with exactly one group
* Use `group_index=i` to select the `i`th instance

In [23]:
(artwork
 .select('ArtistBio')
 .with_columns(country_of_birth = pl.col('ArtistBio').str.extract(r', born ([a-zA-Z]+)'),
               year_of_death = pl.col('ArtistBio').str.extract(r'–(\d{4,4})'))
 .head(5))

ArtistBio,country_of_birth,year_of_death
str,str,str
"""(Austrian, 1841–1918)""",null,"""1918"""
"""(French, born 1944)""",null,null
"""(Austrian, 1876–1957)""",null,"""1957"""
"""(French and Swiss, born Switzerland 1944)""","""Switzerland""",null
"""(Austrian, 1876–1957)""",null,"""1957"""


## <font color="red"> Exercise 3.1.3</font>

**Tasks:**

1. Inspect the `sex_and_age` column
2. Use RegEx to extract the following columns
    a. Gender
    b. age_lower_limit
    c. age_upper_limit

Example based on an example from [Wickim (2014)](https://vita.had.co.nz/papers/tidy-data.pdf)

In [24]:
df = pl.read_csv("./data/TB_bad_stacked.csv")
df.head()

country,group,cases
str,str,f64
"""AD""","""f0-14""",0.0
"""AE""","""f0-14""",4.0
"""AF""","""f0-14""",3500.0
"""AG""","""f0-14""",0.0
"""AI""","""f0-14""",0.0


In [25]:
# Your code here

## Recoding with a `dict` in `polars`

* Use `more_polars.recode(col, d, default=None)`
* `d` is the translation `dict`
* Use `default` keyword to add a default value

In [1]:
from more_polars import recode

new_gender = {'Male':'m', 'Female':'f'}

In [ ]:
(artists
 .select('Gender')
 .with_columns(GenderNew = pl.col('Gender') >> recode(new_gender))
).head()

### Mismatches become `null` by default

In [33]:
(artists
 .select('Gender')
 .with_columns(GenderNew = pl.col('Gender') >> recode(new_gender))
 .groupby(['Gender', 'GenderNew'])
 .agg(pl.col('GenderNew').count().alias('Count'))
).head(14)

Gender,GenderNew,Count
str,str,u32
null,null,3141
"""Non-Binary""",null,2
"""male""",null,15
"""female""",null,1
"""Male""","""m""",9762
"""Non-binary""",null,1
"""Female""","""f""",2300


In [34]:
new_gender = {'male':'m', 'female':'f', 'non-binary':'nb'}

In [35]:
artist_w_recode_gender = (artists
                         .select(pl.col('Gender').str.to_lowercase().alias('Gender'))
                         .with_columns(GenderNew = pl.col('Gender') >> recode(new_gender, default="Unknown"))
)
artist_w_recode_gender.head()

Gender,GenderNew
str,str
"""male""","""m"""
"""male""","""m"""
"""male""","""m"""
"""male""","""m"""
"""male""","""m"""


In [36]:
(artist_w_recode_gender
 .groupby(['Gender', 'GenderNew'])
 .agg(pl.col('GenderNew').count().alias('Count'))
)

Gender,GenderNew,Count
str,str,u32
"""female""","""f""",2301
null,"""Unknown""",3141
"""male""","""m""",9777
"""non-binary""","""nb""",3


## <font color="red"> Exercise 3.1.4</font>

Use `recode` to create a column called `American` in the `artists` data set that will contains `'Yes'` if the artist is American and `'No'` otherwise.  Group and aggregate to get counts for all labels in the new column.

In [37]:
# Your code here